In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

In [2]:
import sys
sys.path.append('/Users/katiehuang/Desktop/metis/projects/onl_ds5_project_2/py')
from loadpage import get_soup

In [3]:
# Create url list (for 19 pages)
listopia_base_url="https://www.goodreads.com/list/show/2451.I_Saw_the_Movie_Read_the_Book"

listopia_url_list=[listopia_base_url]
for i in range(2,20):
    url = listopia_base_url + '?page=' + str(i)
    listopia_url_list.append(url)

In [4]:
# Create soup_list (19 soup for 19 pages)
listopia_soup_list=[]
for url in listopia_url_list:
    listopia_soup_list.append(get_soup(url))

In [5]:
len(listopia_soup_list)

19

In [ ]:
# Find url for each book from the list (I Saw the Movie & Read the Book) webpages
# Try first page (listopia_base_url)

base_book_url="https://www.goodreads.com/"
book_url_list=[]

soup = listopia_soup_list[1]

table= soup.find('table', class_="tableList js-dataTooltip")
rows = [row for row in table.find_all('tr')]

for i in range(100):
    book_url = rows[i].find_all('a')[0].get('href')
    book_url_list.append(base_book_url+book_url)

In [ ]:
# Find url for each book from the list (I Saw the Movie & Read the Book) webpages
# Create a list of url for 1800+ books

book_url_list=[]
base_book_url="https://www.goodreads.com/"

for soup in listopia_soup_list[:-1]:
    table= soup.find('table', class_="tableList js-dataTooltip")
    rows = [row for row in table.find_all('tr')]

    for i in range(100):
        book_url = rows[i].find_all('a')[0].get('href')
        book_url_list.append(base_book_url+book_url)

In [ ]:
len(book_url_list)

In [ ]:
# Create soup_list for the books

book_soup_list=[]
for book_url in book_url_list[0:201]:
    book_soup = get_soup(book_url)
    book_soup_list.append(book_soup)

In [ ]:
len(book_soup_list)

In [ ]:
for book_url in book_url_list[201:901]:
    book_soup = get_soup(book_url)
    book_soup_list.append(book_soup)

In [ ]:
len(book_soup_list)

In [ ]:
for book_url in book_url_list[901:]:
    book_soup = get_soup(book_url)
    book_soup_list.append(book_soup)

In [ ]:
len(book_soup_list)

In [ ]:
# Define function to get book data from the book page
def get_bookdata(book): # book = soup in book_soup_list
    
    headers=['book_title', 'author', 'rating_value', 'rating_count', 'review_count', 'page', 'year']
    
    # Find book title, author, rating_value, rating_count, review_count, page
    title = book.find('h1').text.strip('\n').lstrip()
    author = book.find('a', class_="authorName").text
    rating_value = float(book.find('span', attrs={'itemprop':'ratingValue'}).text.strip('\n').lstrip())
    rating_count = int(book.find('meta', attrs={'itemprop':'ratingCount'}).get('content'))
    review_count = int(book.find('meta', attrs={'itemprop':'reviewCount'}).get('content'))
    page = int(book.find('span', attrs={'itemprop':'numberOfPages'}).text.strip(' pages'))

    # Find year first published
    for div in book.find_all('div', attrs={'id':'details'}):
        for row in div.find_all('nobr', class_='greyText'):
            year = row.text
            year = int(''.join(i for i in year if i.isdigit())[-4:])
            
    
    
    # Find earliest year
    string = div.find_all('div', class_='row')[1].text
    str_clean = string.replace('(',' ').replace(')',' ').split()
    year_list = [int(s) for s in str_clean if s.isdigit()]
    year = min(year_list)
    
    
    book_dict = dict(zip(headers, [title,
                                   author,
                                   rating_value,
                                   rating_count,
                                   review_count,
                                   page,
                                   year]))
    
    return book_dict

In [ ]:
# Define function to get book data from the book page
# Try fixing missing value problem
# Try fixing first published year problem (if there's only first published year)


def get_bookdata(book): # book = soup in book_soup_list
    
    headers=['book_title', 'author', 'rating_value', 'rating_count', 'review_count', 'page', 'year']
    
    # Assign default value
    title, author, rating_value, rating_count, review_count, page, year = \
    np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    # Find book title, author, rating_value, rating_count, review_count, page
    title = book.find('h1').text.strip('\n').lstrip()
    author = book.find('a', class_="authorName").text
    rating_value = float(book.find('span', attrs={'itemprop':'ratingValue'}).text.strip('\n').lstrip())
    rating_count = int(book.find('meta', attrs={'itemprop':'ratingCount'}).get('content'))
    review_count = int(book.find('meta', attrs={'itemprop':'reviewCount'}).get('content'))
    if book.find('span', attrs={'itemprop':'numberOfPages'}) is not None:
        page = int(book.find('span', attrs={'itemprop':'numberOfPages'}).text.strip(' pages'))

    # Find year first published
    for div in book.find_all('div', attrs={'id':'details'}):
        for row in div.find_all('nobr', class_='greyText'):
            year = row.text
            year = int(''.join(i for i in year if i.isdigit())[-4:])
            
    
    
    # Find earliest year
    if div.find_all('div', class_='row') != []:
        string = div.find_all('div', class_='row')[-1].text
        str_clean = string.replace('(',' ').replace(')',' ').split()
        year_list = [int(s) for s in str_clean if s.isdigit()]
        if year_list != []:
            year = min(year_list)
    
    
    book_dict = dict(zip(headers, [title,
                                   author,
                                   rating_value,
                                   rating_count,
                                   review_count,
                                   page,
                                   year]))
    
    return book_dict

In [ ]:
# Test with Harry Potter #1
hp_url = book_url_list[0]
hp = book_soup_list[0]
get_bookdata(hp)

In [ ]:
# Create list of book_dict
book_data=[]
for book_soup in book_soup_list:
    book = get_bookdata(book_soup)
    book_data.append(book)

In [ ]:
len(book_data)

In [ ]:
# Create pd DataFrame of 1800 books!
book_df = pd.DataFrame(book_data)
book_df

In [ ]:
# Save the data to pickle file
book_df.to_pickle('../dump/book_data')